In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input"))

In [ ]:
train = pd.read_csv('../input/train.csv')

In [ ]:
train.head()

In [ ]:
cat_df = train[['Transaction-Type','Company-response','Consumer-disputes']]

In [ ]:
cat_df['Company-response'].fillna('No Response',inplace=True)
cat_df['Consumer-disputes'].fillna('No',inplace=True)

In [ ]:
cat_df['Transaction-Type'] = cat_df['Transaction-Type'].astype('category')
cat_df['Company-response'] = cat_df['Company-response'].astype('category')
cat_df['Consumer-disputes'] = cat_df['Consumer-disputes'].astype('category')

In [ ]:
cat_df_onehot = cat_df.copy()
cat_df_onehot = pd.get_dummies(cat_df_onehot, columns=['Transaction-Type','Company-response','Consumer-disputes'], 
                               prefix = ['Transaction','Response','Disputes'])

In [ ]:
cat_df_onehot.head()

In [ ]:
cat_df_onehot.isnull().sum()

In [ ]:
text_df = train['Complaint-reason']

In [ ]:
text_df.isnull().sum()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
text_df = pd.DataFrame(text_df)

In [ ]:
text_df.columns

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

In [ ]:
text_df_counts = text_df['Complaint-reason'].value_counts()

In [ ]:
text_df_counts['Incorrect information on credit report']

In [ ]:
def text_features(df,df_counts):
    df['word_count'] = df['Complaint-reason'].apply(lambda x: len(str(x).split(" ")))
    df['char_count'] = df['Complaint-reason'].str.len()
    df['avg_word'] = df['Complaint-reason'].apply(lambda x: avg_word(x))
    df['stopwords'] = df['Complaint-reason'].apply(lambda x: len([x for x in x.split() if x in stop]))
    df['sentence_freq'] = df['Complaint-reason'].apply(lambda x: df_counts[x]/len(df))                        

In [ ]:
text_features(text_df,text_df_counts)

In [ ]:
text_df.head()

In [ ]:
text_df_encoded = text_df.drop('Complaint-reason',axis=1)

In [ ]:
text_df_encoded.head()

In [ ]:
text_df_encoded.reset_index(drop=True, inplace=True)
cat_df_onehot.reset_index(drop=True, inplace=True)

In [ ]:
text_df_encoded.shape

In [ ]:
cat_df_onehot.shape

In [ ]:
final_df = pd.concat([cat_df_onehot,text_df_encoded],axis=1)

In [ ]:
final_df.head()

In [ ]:
final_df.isnull().sum()

In [ ]:
from datetime import date

In [ ]:
train['Date-received'] = pd.to_datetime(train['Date-received'])
train['Date-sent-to-company'] = pd.to_datetime(train['Date-sent-to-company'])
train['Days-lag'] = train['Date-sent-to-company'] - train['Date-received']
train['Days-lag'] = train['Days-lag'].apply(lambda x:x.days)

In [ ]:
train['Days-lag'].head()

In [ ]:
num_df = train['Days-lag']

In [ ]:
final_df = pd.concat([final_df,num_df],axis=1)

In [ ]:
final_df.head()

In [ ]:
train['Complaint-Status'] = train['Complaint-Status'].astype('category')
target = train['Complaint-Status'].cat.codes

In [ ]:
target.head()

In [ ]:
target.value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_df, target, test_size=0.33, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=8,random_state=0)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
prediction = clf.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy = accuracy_score(y_test, prediction)

In [ ]:
accuracy

In [ ]:
train_pred = clf.predict(X_train)

In [ ]:
acc = accuracy_score(y_train, train_pred)

In [ ]:
acc

In [ ]:
sample_sub = pd.read_csv('../input/sample_submission.csv')

In [ ]:
sample_sub.head()

In [ ]:
test = pd.read_csv('../input/test.csv')

In [ ]:
test.head()

In [ ]:
cat_df_test = test[['Transaction-Type','Company-response','Consumer-disputes']]

In [ ]:
cat_df_test['Company-response'].fillna('No Response',inplace=True)
cat_df_test['Consumer-disputes'].fillna('No',inplace=True)

In [ ]:
cat_df_test['Transaction-Type'] = cat_df_test['Transaction-Type'].astype('category')
cat_df_test['Company-response'] = cat_df_test['Company-response'].astype('category')
cat_df_test['Consumer-disputes'] = cat_df_test['Consumer-disputes'].astype('category')

In [ ]:
cat_df_onehot2 = cat_df_test.copy()
cat_df_onehot2 = pd.get_dummies(cat_df_onehot2, columns=['Transaction-Type','Company-response','Consumer-disputes'], 
                               prefix = ['Transaction','Response','Disputes'])

In [ ]:
text_df2 = test['Complaint-reason']

In [ ]:
type(text_df2)

In [ ]:
text_df2 = pd.DataFrame(text_df2)

In [ ]:
type(text_df2)

In [ ]:
text_df2_counts = text_df2['Complaint-reason'].value_counts()

In [ ]:
text_features(text_df2,text_df2_counts)

In [ ]:
text_df2.head()

In [ ]:
text_df_encoded2 = text_df2.drop('Complaint-reason',axis=1)

In [ ]:
text_df_encoded2.head()

In [ ]:
text_df_encoded2.reset_index(drop=True, inplace=True)
cat_df_onehot2.reset_index(drop=True, inplace=True)

In [ ]:
final_df = pd.concat([cat_df_onehot2,text_df_encoded2],axis=1)

In [ ]:
final_df.head()

In [ ]:
from datetime import date

In [ ]:
test['Date-received'] = pd.to_datetime(test['Date-received'])
test['Date-sent-to-company'] = pd.to_datetime(test['Date-sent-to-company'])
test['Days-lag'] = test['Date-sent-to-company'] - test['Date-received']
test['Days-lag'] = test['Days-lag'].apply(lambda x:x.days)

In [ ]:
num_df2 = test['Days-lag']

In [ ]:
final_df = pd.concat([final_df,num_df2],axis=1)

In [ ]:
test_pred = clf.predict(final_df)

In [ ]:
test['Complaint-Status'] = test_pred

In [ ]:
test.head()

In [ ]:
def convert_labels(x):
    if x==0:
        return 'Closed'
    if x==1:
        return 'Closed with explanation'
    if x==2:
        return 'Closed with monetary relief'
    if x==3:
        return 'Closed with non-monetary relief'
    if x==4:
        return 'Untimely response'

In [ ]:
test['Complaint-Status'] = test['Complaint-Status'].apply(convert_labels)

In [ ]:
sub_test = test[['Complaint-ID','Complaint-Status']]

In [ ]:
sub_test.head()

In [ ]:
sub_test.to_csv('sub.csv',index=False)